## 정리노트#6

지금까지의 변수는 **스칼라**를 다뤘지만, 머신러닝 데이터로는 벡터나 행렬 등의 **텐서**가 주로 쓰인다.

In [1]:
# 스칼라
import os
import sys

current_dir = os.getcwd()
sys.path.append(current_dir)

import numpy as np
import dezero.functions as F
from dezero import Variable

x = Variable(np.array(1.0))
y = F.sin(x)
print(y)

variable(0.8414709848078965)


In [2]:
# 텐서 
x = Variable(np.array(([[1,2,3],[4,5,6]])))
y = F.sin(x)
print(y)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])


텐서는 위와 같이 sin함수가 원소 각각에 적용된다. 따라서 입력과 출력 텐서의 형상은 바뀌지 않는다. 지금까지의 Dezero 함수들은 원소별 계산이 가능하다.

지금까지의 역전파 구현은 **스칼라**를 대상으로 진행했지만, 텐서를 사용한 계산에 역전파를 적용할 수 없다.<br>
하지만 코드는 문제없이 작용하는데, 그 이유는
- 스칼라를 대상으로 역전파를 구현했고
- 함수에 텐서를 건네면 텐서의 원소마다 '스칼라'로 계산한다.
- 텐서별 원소의 '스칼라' 계산이 이루어지면 스칼라를 가정해 구현한 역전파는 '텐서'의 원소별 계산에서도 성립한다.

In [3]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
t = x + c 
y = F.sum(t) # 마지막 출력은 스칼라 

In [4]:
y.backward(retain_grad = True)
print(y.grad)
print(t.grad)
print(x.grad)
print(c.grad)

variable(1)
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])


이때 기울기의 형상과 데이터(순전파 때의 데이터)의 형상이 일치한다.( 어떤 함수의 입력 변수(데이터)와 그 변수에 대한 기울기(미분값)가 같은 차원과 크기를 갖는다는 것을 의미) <br>
역전파를 진행할 때, 출력값 y의 손실 함수로부터의 기울기를 dy라고 하면, 이 기울기를 이용하여 입력 x의 기울기 dx를 계산한다. 이때 dx는 x의 원래 형상 (m, n)을 그대로 유지합니다. 즉, 각 입력 변수 x의 각 원소에 대해, **해당 원소가 최종 출력 y에 얼마나 영향을 미쳤는지**를 나타내는 기울기가 계산된다.<br>
기울기를 계산할 때 각 입력 변수의 영향을 정확하게 파악하고, 각 변수에 대한 손실 함수의 민감도를 적절히 업데이트하기 위해 필요하다.<br>
<br>
즉 x.shape == x.grad.shape이며 c.shape == c.grad.shape 이고 t.shape == t.grad.shape이다. 이 성질을 이용하여 원소별 계산이 아닌 함수를 구현할 수 있다.

### 형상 변환 함수

텐서의 형상을 변환하는 reshape함수와 행렬을 전치하는 transpose 함수는 모두 텐서의 형상을 바꾸는 함수이다.

+ "계산을 원소별로 하지 않는다"는 표현은 주로 대규모 데이터를 다룰 때 사용되는 개념으로, 개별 원소들에 대해 하나씩 연산을 수행하는 대신에, 여러 원소에 대한 연산을 동시에, 그리고 더 추상화된 형태로 수행하는 것을 말한다. 이는 주로 벡터화(vectorization), 행렬 연산, 혹은 텐서 연산 등을 통해 구현된다. 이러한 방식은 컴퓨터가 수행하기에 더 최적화되어 있고, 실행 시간을 대폭 줄여줄 수 있다.

- 텐서 연산을 수행할 때, 입력 텐서들의 형상이 호환되어야 연산이 가능하다. 예를 들어, 행렬 곱셈에서는 첫 번째 행렬의 열 수와 두 번째 행렬의 행 수가 일치해야 한다. 또한, 원소별 연산(element-wise operation)을 수행할 때는 두 텐서의 형상이 완전히 동일하거나 브로드캐스팅 규칙에 따라 조정될 수 있어야 한다.

In [5]:
# reshape 함수 구현

import numpy as np

x = np.array([[1,2,3],[4,5,6]])
y = np.reshape(x, (6,)) # x의 형상을 (2,3)에서 (6,)=6개의 요소를 가진 1차원 배열로 만들겠다는 의미
print(y)

[1 2 3 4 5 6]


reshape 함수는 단순히 형상만 변환하고 구체적인 계산은 아무것도 하지 않는다. 따라서 역전파는 출력 쪽에서 전해지는 기울기를 그대로 입력 쪽으로 흘려보내는데, 기울기의 형상이 입력의 형상과 같아지도록 변환해 준다.

**functions 파일에 추가 - Reshape 클래스**
```python
class Reshape(Function):
    def __init__(self, shape):
        self.shape = shape
    
    def forward(self, x):
        self.x_shape = x.shape  #입력 x의 원래 모양을 저장
        y = x.reshape(self.shape)  #입력 x를 self.shape의 형태로 재배열
        return y
    
    def backward(self, gy):
        return reshape(gy, self.x_shape)  #기울기 gy를 원래 x의 모양으로 재배열
    
from dezero.core import as_variable

def reshape(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return Reshape(shape)(x)
```

In [7]:
import numpy as np
import dezero.functions as F
from dezero import Variable

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.reshape(x, (6,))
y.backward(retain_grad=True)
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


넘파이의 reshape 함수는 ndarray 인스턴스의 메서드로 사용할 수 있다. 또한 가변 인수도 받는다.<br>
Variable 클래스에 <br>
```python
def reshape(self,*shape):
    if len(shape) == 1 and isinstance(shape[0], (tuple,list)):
        shape = shape[0]
       return dezero.functions.reshape(self, shape)
```
가변 길이 인수를 받는 코드를 추가해 준다.

### 행렬의 전치

이번 단계에서는 입력 변수가 행렬(2차원 텐서)일 때로 한정하여 구현한다.

In [8]:
x = np.array([[1,2,3],[4,5,6]])
y = np.transpose(x)
print(y)

[[1 4]
 [2 5]
 [3 6]]


x의 형상이 (2,3)에서 (3,2)로 달라진다.

**transpose 함수를 구현**

```python
class Transpose(Function):
    def forward(self,x):
        y = np.transpose(x)
        return y 
    
    def backward(self,gy):
        gx = transpose(gy)
        return gx
    
def transpose(x):
    return Transpose()(x)
```

In [10]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.transpose(x)
y.backward()
print(x.grad)

variable([[1 1 1]
          [1 1 1]])


넘파이의 reshape 함수는 ndarray 인스턴스의 메서드로 사용할 수 있다. 또한 가변 인수도 받는다.
Variable 클래스에

```python
def transpose(self):
    return dezero.functions.transpose(self)
    
    @property
    def T(self):
        return dezero.functions.transpose(self)
```
가변 길이 인수를 받는 코드를 추가해 준다.

return dezero.functions.transpose(self)는 '인스턴스 메서드'로 이용하기 위한 코드이다.<br>
두 번째 T에는 @property 데코레이터가 붙어 있는데, '인스턴스 변수'로 사용할 수 있게 해주는 데코레이터다. T라는 프로퍼티는 @property 데코레이터를 통해 transpose() 메서드의 결과를 속성처럼 쉽게 접근할 수 있게 해 주고, .T를 호출할 때마다 메서드를 호출하는 것처럼 작동하여 계산된 전치 행렬을 얻을 수 있다.

### 합계  함수
sum 함수 역전파에서는 입력 변수의 형상과 같아지도록 기울기의 원소를 복사한다.

**functions.py 파일에 추가**
```python
class Sum(Function):
    def forward(self, x):
        self.x_shape = x.shape
        y = x.sum()
        return y
    
    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx
    
def sum(x):
    return Sum(x)
```

In [12]:
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([1,2,3,4,5,6]))
y = F.sum(x)
y.backward()

print(y)
print(x.grad)

variable(21)
variable([1 1 1 1 1 1])


In [13]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x)
y.backward()

print(y)
print(x.grad)

variable(21)
variable([[1 1 1]
          [1 1 1]])


위 결과를 보면 2차원 배열을 입력했을 때 x.grad와 x가 같은 형상임을 알 수 있다.

넘파이의 np.sum 함수는 합계를 구할 때 **축**을 지정할 수 있다. 

In [14]:
x = np.array([[1,2,3],[4,5,6]])
y = np.sum(x, axis=0)
print(y)
print(x.shape, '->', y.shape)

[5 7 9]
(2, 3) -> (3,)


**functions.py 파일에 추가** <br>
```python
from dezero import utils

class Sum(Function):
    def __init__(self, axis, keepdims):
        self.axis = axis
        self.keepdims = keepdims # keepdims는 입력과 출력의 차원수를 똑같게 유지할지 정하는 플래그이다. 
        
    def forward(self, x):
        self.x_shape = x.shape
        y = x.sum(axis=self.axis, keepdims=self.keepdims)
        return y
    
    def backward(self, gy):
        gy = utils.reshape_sum_backward(gy, self.x_shape, self.axis, self.keepdims)
        gx = broadcast_to(gy, self.x_shape)
        return gx
    
def sum(x, axis=None, keepdims=False):
    return Sum(axis, keepdims)(x) 
```

Sum 클래스를 초기화할 때 axis와 keepdims를 입력받아 속성으로 설정한다.

utils.reshape_sum_backward 함수는 gy의 형상을 **미세하게 조정**한다.

In [16]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x, axis = 0)
y.backward()
print(y)
print(x.grad)

x = Variable(np.random.randn(2,3,4,5))
y = x.sum(keepdims=True)
print(y.shape)

variable([5 7 9])
variable([[1 1 1]
          [1 1 1]])
(1, 1, 1, 1)


### 브로드캐스트 함수

넘파이의 np.broadcast_to(x, shape) 함수는 인스턴스 ndarray 인스턴스인 x의 원소를 복제하여 shape 인수로 지정한 형상이 되도록 해준다. 

In [17]:
import numpy as np 
x = np.array([1,2,3])
y = np.broadcast_to(x, (2,3))
print(y)

[[1 2 3]
 [1 2 3]]


In [18]:
import numpy as np
from dezero.utils import sum_to

x = np.array([[1,2,3],[4,5,6]])
y = sum_to(x, (1,3))
print(y)

y = sum_to(x, (2, 1))
print(y)

[[5 7 9]]
[[ 6]
 [15]]


sum_to(x, shape) 함수는 shape 형상이 되도록 합을 계산한다.

```python
class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape
        
    def forward(self, x): # 입력 x의 현재 모양을 저장하고, np.broadcast_to 함수를 사용하여 입력 x를 목표 shape으로 브로드캐스팅
        self.x_shape = x.shape
        y = np.broadcast_to(x, self.shape)
        return y
        
    def backward(self, gy): # 역전파 시, 입력된 그래디언트 gy를 원래 x의 모양인 self.x_shape으로 변환
        gx = sum_to(gy, self.x_shape)
        return gx

def broadcast_to(x, shape):
    if x.shape == shape: # x의 현재 모양이 이미 목표 shape와 동일하다면, 추가적인 처리 없이 x를 반환
        return as_variable(x)
    return BroadcastTo(shape)(x)
```

```python
class SumTo(Function):
    def __init__(self, shape):
        self.shape = shape

    def forward(self, x):
        self.x_shape = x.shape
        y = utils.sum_to(x, self.shape) # utils.sum_to 함수를 사용하여 데이터를 목표 shape로 축소
        return y

    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape) # broadcast_to 함수를 사용하여 gy를 원래 입력 x의 모양으로 확장하고, 이 결과인 gx를 반환
        return gx


def sum_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return SumTo(shape)(x)
```

위 코드처럼 broadcast함수와 sum_to 함수는 상호의존적이다.

순전파는 ndarray 인스턴스를 사용해 구현했기 때문에 브로드캐스트가 일어나지만 역전파에서는 브로드캐스트의 역전파가 일어나지 않는다.<br>
넘파이 브로드캐스트는 broadcast_to 함수에서 이루어지고 broadcast_to 함수의 역전파는 sum_to 함수에 해당한다. <br>
```python
class Add(Function):
    def forward(self, x0, x1):
        # 입력 배열들의 형태를 인스턴스 변수 self.x0_shape와 self.x1_shape에 저장하여 역전파 때 사용
        self.x0_shape, self.x1_shape = x0.shape, x1.shape
        y = x0 + x1 # 서로 다른 모양의 배열이 주어지면, 작은 배열이 큰 배열의 형태에 맞추어 자동으로 확장되어 덧셈이 수행
        return y

    def backward(self, gy):
        gx0, gx1 = gy, gy
        # 두 입력 배열의 모양이 다른 경우, 즉 브로드캐스팅이 발생한 경우에는 각 입력 배열에 맞게 그라디언트의 모양을 조정
        if self.x0_shape != self.x1_shape:  
            gx0 = dezero.functions.sum_to(gx0, self.x0_shape) # gx0의 모양을 x0의 원래 모양으로 축소
            gx1 = dezero.functions.sum_to(gx1, self.x1_shape) # gx1의 모양을 x1의 원래 모양으로 축소
        return gx0, gx1
```
두 형상이 다를 때 브로드캐스트용 역전파를 계산할 수 있게 기울기 gx()는 x()의 형상이 되도록 합을 구하고, 마찬가지로 기울기 gx1은 x1의 형상이 되도록 합을 구한다.

In [23]:
import numpy as np
from dezero import Variable

x0 = Variable(np.array([1, 2, 3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)

y.backward()
print(x1.grad)

variable([11 12 13])
variable([3])


### 백터의 내적과 행렬의 곱

백터의 내적은 백터 사이의 대응 원소의 곱을 모두 합한 값이고, 행렬의 곱은 왼쪽 행렬의 가로 방향 벡터와 오른쪽 행렬의 세로 방향 벡터 사이의 내적을 계산한다. 백터의 내적과 행렬의 곱 계산은 모두 np.dot 함수로 처리할 수 있다.<br>
np.dot(x,y)의 두 인수가 모두 1차원 배열이면 백터의 내적을 계산하고, 인수가 2차원 배열이면 행렬의 곱을 계산한다.

행렬과 벡터를 사용한 계산시 주의할 점<br>
▪ 형상(shape)에 주의<br>
▪ 행렬 a와 b의 대응하는 차원(축)의 원소 수가 일치해야 한다.<br>
▪ 결과로 만들어진 행렬 c의 형상은 행렬 a와 같은 수의 행을, 행렬 b와 같은 수의 열을 갖는다.

```python
class MatMul(Function):
    def forward(self, x, W):
        y = x.dot(W) #  x와 W의 행렬 곱을 계산. 이는 넘파이의 dot 함수를 사용하여 수행
        return y

    def backward(self, gy):
        x, W = self.inputs
        gx = matmul(gy, W.T) # gx는 gy와 W의 전치 행렬(W.T)과의 행렬 곱으로 계산. 입력 x에 대한 기울기다.
        gW = matmul(x.T, gy) # x의 전치 행렬(x.T)과 gy의 행렬 곱으로 계산. 가중치 W에 대한 기울기다.
        return gx, gW 


def matmul(x, W):
    return MatMul()(x, W)
```

위 함수를 사용하여 아래의 계산이 가능해진다.

In [26]:
x = Variable(np.random.randn(2, 3))
w = Variable(np.random.randn(3, 4))
y = F.matmul(x, w)
y.backward()

print(x.grad.shape)
print(w.grad.shape)

(2, 3)
(3, 4)
